In [34]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import pandas as pd
import json
import geopandas as gpd
from io import StringIO
import numpy as np
import statsmodels.api as sm

ModuleNotFoundError: No module named 'geopandas'

In [9]:
with open('../data/api_toxen.json') as fi:
    credentials = json.load(fi)
    
api_key = credentials['api_key']

In [10]:
endpoint = 'https://data.nashville.gov/resource/2u6v-ujjs.geojson'

In [11]:
params = {'offense_description':'BURGLARY- AGGRAVATED',  '$where': 'incident_reported between \'2021-01-01\' and \'2021-06-30\'','$limit': 10000}

In [12]:
response = requests.get(endpoint, params)
response

<Response [200]>

In [13]:
BA_gdf = gpd.read_file(StringIO(response.text))
BA_gdf.columns

NameError: name 'gpd' is not defined

# 2

In [ ]:
ctract_gdf = gpd.read_file("../data/tl_2019_47_tract.shp")
ctract_gdf = ctract_gdf.to_crs(epsg=4326)
ctract_gdf.rename(columns= {'TRACTCE': 'tract'}, inplace=True)
ctract_gdf

In [ ]:
BA_gdf = BA_gdf[
                ['victim_number',
                'incident_occurred', 
                'incident_number',
                'geometry']
                                    ]
                 

In [14]:
ctract_gdf = ctract_gdf[
                        ['tract',
                        'geometry']
]

NameError: name 'ctract_gdf' is not defined

In [15]:
ab_by_censustract = gpd.sjoin(BA_gdf, ctract_gdf, op = 'within')
ab_by_censustract

NameError: name 'gpd' is not defined

In [16]:
ab_by_censustract_drop = ab_by_censustract.drop_duplicates(subset=['incident_number'])
ab_by_censustract_drop.groupby('tract').count().nlargest(2,'victim_number')
                                                            

NameError: name 'ab_by_censustract' is not defined

Census Tract 163 had the highest number of aggrevated burglaries at 53.

In [17]:
ab_by_censustract

NameError: name 'ab_by_censustract' is not defined

# 3

In [18]:
with open('../data/api_census_token.json') as fi:
    credentials = json.load(fi)
    
api_key_1 = credentials['api_key']

In [19]:
endpoint_1 = 'https://api.census.gov/data/2019/acs/acs5'

In [20]:
parameters_1 = {'get': 'B01001_001E', 'for': 'tract', 'in': 'state:47 county:037', 'key': api_key_1}

In [21]:
response_1 = requests.get(endpoint_1, parameters_1)
response_1

<Response [200]>

In [22]:
census_df= pd.DataFrame(response_1.json())
census_df.columns = census_df.iloc[0]
census_df = census_df.iloc[1:,:]
census_df.rename(columns = {'B01001_001E': 'population'}, inplace=True)

In [23]:
endpoint_2 = 'https://api.census.gov/data/2019/acs/acs5/subject'

In [24]:
parameters_2 = {'get': 'S1901_C01_012E', 'for': 'tract', 'in': 'state:47 county:037', 'key': api_key_1}

In [25]:
response_2 = requests.get(endpoint_2, parameters_2)
response_2

<Response [200]>

In [26]:
census_df_subject = pd.DataFrame(response_2.json())
census_df_subject.columns = census_df_subject.iloc[0]
census_df_subject = census_df_subject.iloc[1:,:]
census_df_subject.rename(columns= {'S1901_C01_012E': 'median_income'}, inplace=True)
census_df_subject

,median_income,state,county,tract
1,75579,47,037,015631
2,45129,47,037,015804
3,134786,47,037,017701
4,107813,47,037,017702
5,87591,47,037,018301
...,...,...,...,...
157,37656,47,037,015613
158,54149,47,037,015620
159,48221,47,037,016100
160,50857,47,037,011400


In [27]:
complete_census_df = pd.merge(census_df, census_df_subject, how='inner', on=['state', 'county', 'tract'])
complete_census_df = complete_census_df[['population', 'tract', 'median_income']]
complete_census_df.shape

(161, 3)

In [28]:
ultimate_df = ab_by_censustract.merge(complete_census_df, on="tract")
ultimate_df['population'] = pd.to_numeric(ultimate_df['population'],errors = 'coerce')
ultimate_df.info()

NameError: name 'ab_by_censustract' is not defined

# 4

In [ ]:
ultimate_df = ultimate_df.drop_duplicates(subset=['incident_number'])
ultimate_df_count = ultimate_df['tract'].value_counts().reset_index()
ultimate_df_count.rename(columns = {"index": 'tract', 'tract': 'tot_agg_burg'}, inplace=True)
ultimate_df_2 = ultimate_df.merge(ultimate_df_count, on='tract')
ultimate_df_2.drop(columns = 'victim_number', inplace=True)
ultimate_df_2.drop_duplicates(['population', 'median_income'], inplace=True)
ultimate_df_2['tot_agg_burg'].astype(float)
ultimate_df_2['burglaries_per_1000'] = (ultimate_df_2['tot_agg_burg'] / (ultimate_df_2['population'] / 1000))
ultimate_df_3 = ultimate_df_2.drop(columns=[ 'incident_occurred', 'incident_number', 'index_right'])
ultimate_df_3 = ultimate_df_3.fillna(0)
ultimate_df_3 = ultimate_df_3[ultimate_df_3['population'] != 0]
ultimate_df_3.shape

In [29]:
chloropleth = gpd.sjoin(ctract_gdf, ultimate_df_3, how = 'left')
chloropleth = chloropleth.drop(columns = ['tract_right', 'index_right'])
chloropleth.rename(columns = {'tract_left': 'tract'}, inplace=True)
chloropleth = chloropleth.dropna()
leg_kwds = {'title': 'burglaries_per_1000', 'loc': 'upper left', 
            'bbox_to_anchor': (1, 1.03), 'ncol': 2}

chloropleth.plot(column = 'burglaries_per_1000', figsize=(10, 10),
                 edgecolor = 'black',
                 legend = True, legend_kwds = leg_kwds, 
                 cmap = 'Set2')
plt.show()

NameError: name 'gpd' is not defined

In [30]:
chloropleth['burglaries_per_1000'].unique()

NameError: name 'chloropleth' is not defined

# 5

In [31]:
ultimate_df_3

NameError: name 'ultimate_df_3' is not defined

In [32]:
ultimate_df_3['log_population'] = np.log(ultimate_df_3['population'])
ultimate_df_3

NameError: name 'np' is not defined

In [33]:
poisreg_population = (sm.GLM(endog = ultimate_df_3['tot_agg_burg'],
                             exog = sm.add_constant(ultimate_df_3[['log_population']]),
                             family = sm.families.Poisson())
                     .fit()
                     )

NameError: name 'sm' is not defined